In [ ]:
# Listing all the imports
! pip install imutils
! mkdir /kaggle/working/trained_images/
! rm -rf /kaggle/working/trained_images/
! mkdir /kaggle/working/trained_images/
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import imutils
import math

In [ ]:
! ls /kaggle/working
! ls /kaggle/working/trained_images
!ls -l /kaggle/working/trained_images/ | egrep -c '^-' 

In [ ]:
# image height and image width ----> GLOBAL
img_ht = 256
img_wd = 256

def displayImage(display_name, image):
    cv2.namedWindow(display_name,cv2.WINDOW_AUTOSIZE)
    cv2.imshow(display_name, image)

def findContourEye(thresh_image):
    cnts = cv2.findContours(thresh_image.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = max(cnts, key=cv2.contourArea)
    return cnts

def findContourEyeExtreme(cnts):
    # Locating extreme points on all 4 sides
    leftmost = tuple(cnts[cnts[:,:,0].argmin()][0])
    rightmost = tuple(cnts[cnts[:,:,0].argmax()][0])
    topmost = tuple(cnts[cnts[:,:,1].argmin()][0])
    bottommost = tuple(cnts[cnts[:,:,1].argmax()][0])
    # Locating the top left and bottom right corner
    x1 = leftmost[0]
    y1 = topmost[1]
    x2 = rightmost[0]
    y2 = bottommost[1]
    return x1,y1,x2,y2 

def findRadiusAndCentreOfContourEye(cnts):
    M = cv2.moments(cnts)
    if( M["m00"]==0):
        cX, cY = 0, 0
    else:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    if(cX < cY):
        r = cX
    else:
        r = cY
    return cX,cY,r

def drawCentreOnContourEye(image,cnts,cX,cY):
    cv2.drawContours(image, [cnts], -1, (0, 255, 0), 2)
    cv2.circle(image, (cX, cY), 7, (255, 255, 255), -1)
    cv2.putText(image, "center", (cX - 20, cY - 20),
    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)
    return image
    
def Radius_Reduction(img,cX,cY,r):
    h,w,c=img.shape
    Frame=np.zeros((h,w,c),dtype=np.uint8)
    cv2.circle(Frame,(int(cX),int(cY)),int(r), (255,255,255), -1)
    Frame1=cv2.cvtColor(Frame, cv2.COLOR_BGR2GRAY)
    img1 =cv2.bitwise_and(img,img,mask=Frame1)
    return img1

def imageResize(image, ht, wd):
    # resized_image = imutils.resize(image, height = ht, width = wd)
    resized_image = cv2.resize(image,(wd,ht))
    return resized_image

def crop_black(image):
    org = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(blurred, 10, 255, cv2.THRESH_BINARY)[1]
    # displayImage('thresh',thresh)
    cnts = findContourEye(thresh)
    x1,y1,x2,y2 = findContourEyeExtreme(cnts)
    # print(x1,y1,x2,y2)
    crop = org[y1:y2, x1:x2]
    crop = imageResize(crop, img_ht, img_wd)
    # displayImage("cr1",crop)
    return crop

def imageAugmentation(image):
    x_flip = cv2.flip( image, 0 )
    y_flip = cv2.flip( image, 1 )
    xy_flip = cv2.flip(x_flip,1)
    return x_flip, y_flip, xy_flip

def imageHistEqualization(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return final

def subtract_median_bg_image(im):
    k = np.max(im.shape)//20*2+1
    bg = cv2.medianBlur(im, k)
    sub_med = cv2.addWeighted (im, 4, bg, -4, 100)
    return sub_med

def colorEnhancement(image1,image2):
    image_final = cv2.bitwise_and(image1,image2)
    return image_final

def imageAugSave(path,img1,img2,img3,img4,img_ht,img_wd):
    count = len(os.listdir(path))

    img1 = imageResize(img1, img_ht, img_wd)
    img2 = imageResize(img2, img_ht, img_wd)
    img3 = imageResize(img3, img_ht, img_wd)
    img4 = imageResize(img4, img_ht, img_wd)

    cv2.imwrite(os.path.join(path , '%d.png'%(count+1)), img1)
    cv2.imwrite(os.path.join(path , '%d.png'%(count+2)), img2)
    cv2.imwrite(os.path.join(path , '%d.png'%(count+3)), img3)
    cv2.imwrite(os.path.join(path , '%d.png'%(count+4)), img4)
    return count+1,count+2,count+3,count+4

def processed_test_save(path,img,img_ht,img_wd):
    count = len(os.listdir(path))
    img = imageResize(img,img_ht,img_wd)
    cv2.imwrite(os.path.join(path , '%d.png'%(count+1)), img)
    return count+1

In [ ]:
img_ht = 256
img_wd = 256
path_toCollect =  '/kaggle/input/aptos2019-blindness-detection/train_images'
path_toSave = '/kaggle/working/trained_images'
train_data = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/train.csv')
newDataframe_cols = ['id_code','diagnosis'] 
trained_data = pd.DataFrame(columns=newDataframe_cols)

In [ ]:
def feedToPipeline(image_name,diagnosis_type):
    global path_toCollect
    global path_toCollect
    global img_ht,img_wd
    global trained_data, train_data

    try:
        image_name = str(image_name) + '.png'
        image = cv2.imread(os.path.join(path_toCollect,image_name))
        image = imageResize(image, img_ht, img_wd)
        org_copy = image.copy()
        image_crop = crop_black(image)
        image_clahe = imageHistEqualization(image_crop)
        sub_med = subtract_median_bg_image(image_clahe)
        image_final = colorEnhancement(sub_med, image_clahe)
        aug1, aug2, aug3 = imageAugmentation(image_final)
        count1,count2,count3,count4 = imageAugSave(path_toSave,image_final, aug1, aug2, aug3,img_ht,img_wd)
        count1 = str(count1) + '.png'
        count2 = str(count2) + '.png'
        count3 = str(count3) + '.png'
        count4 = str(count4) + '.png'
        len_trained_data = len(trained_data)
        trained_data.loc[len_trained_data]   = [count1,diagnosis_type] 
        trained_data.loc[len_trained_data+1] = [count2,diagnosis_type] 
        trained_data.loc[len_trained_data+2] = [count3,diagnosis_type] 
        trained_data.loc[len_trained_data+3] = [count4,diagnosis_type]
#         print("Processed")
    except:
        print("+========================+")
        pass

In [ ]:
start = time.time()

# # Vectorize approach took 846 seconds and the for loop took 905 seconds to process more than 3 thousand images
# # 
# # np.vectorize(feedToPipeline)(train_data['id_code'],train_data['diagnosis'])
# # 
from tqdm.notebook import tqdm
for i in tqdm(range(len(train_data))): 
# for i in tqdm(range(1)): 
#     print(i)
    feedToPipeline(train_data['id_code'][i],train_data['diagnosis'][i])
# # 
trained_data.to_csv('/kaggle/working/final_trained.csv',index = False)


In [ ]:
!ls /kaggle/working/
# !ls /kaggle/working/trained_images
!ls -l /kaggle/working/trained_images/ | egrep -c '^-' 